### 자연어 분류예측 III (LSTM과 CNN의 조합)

다음은 시퀀스 to 벡터의 예이다.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets.imdb import load_data, get_word_index       # imdb 데이터. 
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Embedding, Conv1D, MaxPool1D, Dropout
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.optimizers import Adam, RMSprop, SGD
import matplotlib.pyplot as plt
%matplotlib inline

#### 데이터를 불러온다

In [ ]:
n_words = 2000                                        # "사전"의 크기.
(X_train, y_train), (X_test, y_test) = load_data(num_words = n_words)

In [ ]:
# 크기 확인.
print("-"*50)
print("Training data X shape: {}".format(X_train.shape))
print("Training data y shape: {}".format(y_train.shape))
print("-"*50)
print("Test data X shape: {}".format(X_test.shape))
print("Test data y shape: {}".format(y_test.shape))
print("-"*50)

#### 데이터를 탐구해 본다

In [ ]:
# y 변수에서의 유형의 가짓수 = 2임을 확인!
# 바이너리 변수이다!
pd.Series(y_train).unique()

In [ ]:
# X의 값 하나만 가져와 본다. 리스트인 것을 알 수 있다.
# 문자열의 단어 하나 하나가 숫자로 변환되어 있다.
print(X_train[0])

In [ ]:
# 첫 100개의 원소 (리스트)의 길이를 본다. 
# 제각각 임을 알 수 있다.
print([len(a) for a in X_train[0:100]])

In [ ]:
# 변환 사전을 내려받는다.
my_dict = get_word_index(path='imdb_word_index.json')

In [ ]:
# 다음을 실행해서 사전의 내용을 볼 수도 있다.
# my_dict

In [ ]:
# key와 value의 역할을 바꾼다.
my_dict_inv = {v:k for k,v in my_dict.items()}

In [ ]:
# 개개 리뷰의 내용을 본다.
i_article = 10
article = list(pd.Series(X_train[i_article]).apply(lambda x: my_dict_inv[x]))
' '.join(article)

#### 데이터의 전처리

In [ ]:
# 패딩: 모든 리뷰의 길이를 maxlen으로 맞춘다.
# maxlen보다 더 길으면 자르고 maxlen보다 짧으면 0으로 채워 넣는다.
X_train = sequence.pad_sequences(X_train, maxlen = 100)
X_test = sequence.pad_sequences(X_test, maxlen = 100)

#### 모형정의

In [ ]:
hold_prob = 0.5
n_filters = 32                    # CNN 필터의 개수.
k_size = 4                        # CNN 필터의 크기. 
stride_size = 1                   # 스트라이드.
n_neurons = 50                    # 셀별 뉴런의 개수.
n_input = 100                     # Embeding 공간의 차원.
n_pool_size = 2                   # 풀링의 커널 사이즈.

In [ ]:
# LSTM + CNN 모형.
my_model = Sequential()
my_model.add(Embedding(n_words,n_input))           # 최대 단어의 개수, Embedding공간의 차원.
my_model.add(Conv1D(filters=n_filters, kernel_size = k_size, strides=stride_size,padding='valid',activation='relu'))
my_model.add(MaxPool1D(pool_size = n_pool_size))
my_model.add(LSTM(units=n_neurons, return_sequences=False, input_shape=(None, n_input), activation='tanh'))
my_model.add(Dropout(rate=hold_prob))
my_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# 모형의 구조를 본다.
my_model.summary()

#### 손실함수 정의, 최적화 방법 정의

In [ ]:
n_epochs = 5                        # 에포크의 총 길이.
n_batch_size = 24                   # 배치의 크기.
learn_rate = 0.005                  # 학습속도.  

In [ ]:
# 최적화 방법 정하고 컴파일.
my_optimizer=Adam(lr=learn_rate)
my_model.compile(loss = "binary_crossentropy", optimizer = my_optimizer, metrics=["accuracy"])

#### 학습 실행. 정확도의 변화를 시각화 한다.  

In [ ]:
my_summary = my_model.fit(X_train, y_train, epochs=n_epochs, batch_size = n_batch_size, validation_split=0.2, verbose = 1)

In [ ]:
plt.plot(my_summary.history['acc'], c="b")
plt.plot(my_summary.history['val_acc'], c="g")
plt.title('Training History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

#### 예측 및 결과 평가 (test Accuracy 계산)

In [ ]:
ACC = my_model.evaluate(X_test, y_test, verbose=0)[1]
np.round(ACC,3)